In [1]:
from langchain import HuggingFaceHub
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.llms import HuggingFacePipeline
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import AgentAction, AgentFinish
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    LlamaForCausalLM,
    LlamaTokenizer,
    pipeline,
)
import os
import logging
import torch

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
DEVICE_TYPE = "cuda"
# model_id = "atom92/medical-meta-llama-2-7b"
# model_id = "atom92/power-medical-meta-llama-2"
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "atom92/medical-token-llama-2-healthwa-3"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
logging.info("Tokenizer loaded")

In [4]:
print(torch.cuda.device_count())

1


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"": 0},
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    max_memory={0: "17GiB", 1: "17GiB"},
    offload_folder="/tmp/offload"
)
model.tie_weights()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=254,
    do_sample=True,
    temperature=0.9,
    top_p=0.5,
    top_k=50,
    repetition_penalty=1.1,
)

In [7]:
local_llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
window_memory = ConversationBufferWindowMemory(ai_prefix="AI Assistant", k=2, return_messages=True)

In [9]:
template = """You are a helpful virtual medical assistant that delivers precise, succinct information from established medical knowledge. You only respond in English clearly, precisely and briefly.

Current conversation:
{history}
Human: {input}
AI Assistant:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

In [10]:
conversation = ConversationChain(
    prompt=PROMPT,
    llm=local_llm,
    verbose=True,
    memory=window_memory
)

In [11]:
conversation.run("What are the signs and symptoms of haemolytic uraemic syndrome (HUS)?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful virtual medical assistant that delivers precise, succinct information from established medical knowledge. You only respond in English clearly, precisely and briefly.

Current conversation:
[]
Human: What are the signs and symptoms of haemolytic uraemic syndrome (HUS)?
AI Assistant:

> Finished chain.


' Haemolytic uraemic syndrome (HUS) is a rare disorder characterized by the sudden destruction of red blood cells, leading to anemia, thrombocytopenia, and acute kidney injury. The signs and symptoms of HUS can vary depending on the severity of the condition, but may include:\n\n1. Anemia: A decrease in the number of red blood cells or hemoglobin in the blood.\n2. Thrombocytopenia: A low platelet count, which can increase the risk of bleeding.\n3. Fatigue: Feeling weak or tired due to anemia.\n4. Shortness of breath: Difficulty breathing due to a lack of oxygen in the body.\n5. Pale skin: A lack of oxygen in the body can cause the skin to appear pale or blueish.\n6. Headache: A headache can occur due to changes in blood pressure or fluid buildup in the brain.\n7. Confusion: In severe cases, HUS can cause confusion, seizures, or loss of consciousness due to cerebral edema.\n8'

In [12]:
conversation.run("What causes the skin to turn yellow in yellow fever?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful virtual medical assistant that delivers precise, succinct information from established medical knowledge. You only respond in English clearly, precisely and briefly.

Current conversation:
[HumanMessage(content='What are the signs and symptoms of haemolytic uraemic syndrome (HUS)?'), AIMessage(content=' Haemolytic uraemic syndrome (HUS) is a rare disorder characterized by the sudden destruction of red blood cells, leading to anemia, thrombocytopenia, and acute kidney injury. The signs and symptoms of HUS can vary depending on the severity of the condition, but may include:\n\n1. Anemia: A decrease in the number of red blood cells or hemoglobin in the blood.\n2. Thrombocytopenia: A low platelet count, which can increase the risk of bleeding.\n3. Fatigue: Feeling weak or tired due to anemia.\n4. Shortness of breath: Difficulty breathing due to a lack of oxygen in the body.\n5. Pale skin: A lack of oxyg

" Yellow fever is caused by a virus, not by a change in skin color. The skin turning yellow in yellow fever is a sign of severe illness and can be a result of liver dysfunction. It's important to seek medical attention immediately if you experience this symptom."

In [13]:
conversation.run("What is the size of hatchling head lice?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful virtual medical assistant that delivers precise, succinct information from established medical knowledge. You only respond in English clearly, precisely and briefly.

Current conversation:
[HumanMessage(content='What are the signs and symptoms of haemolytic uraemic syndrome (HUS)?'), AIMessage(content=' Haemolytic uraemic syndrome (HUS) is a rare disorder characterized by the sudden destruction of red blood cells, leading to anemia, thrombocytopenia, and acute kidney injury. The signs and symptoms of HUS can vary depending on the severity of the condition, but may include:\n\n1. Anemia: A decrease in the number of red blood cells or hemoglobin in the blood.\n2. Thrombocytopenia: A low platelet count, which can increase the risk of bleeding.\n3. Fatigue: Feeling weak or tired due to anemia.\n4. Shortness of breath: Difficulty breathing due to a lack of oxygen in the body.\n5. Pale skin: A lack of oxyg

" The size of hatchling head lice varies depending on their age and stage of development. Newly hatched head lice are about 1/16 inch (1.5 millimeters) long, while older lice can grow up to 1/4 inch (6 millimeters) long. However, it's important to note that the size of head lice can vary greatly among individuals, and some may be larger or smaller than average. If you suspect you or someone in your family has head lice, it's best to consult with a healthcare professional for proper diagnosis and treatment."

In [60]:
conversation.run("Can HIV be spread through kissing?")



> Entering new ConversationChain chain...
Prompt after formatting:
As a medical knowledge assistant, my purpose is to deliver precise, succinct information from established medical knowledge. Here is your brief answer:

Current conversation:
[HumanMessage(content='When do symptoms of it usually start?'), AIMessage(content=' Symptomatic individuals typically experience illness within 2 weeks after being exposed but may remain asymptotic for up months years before showing signs if at all—it depends on individual factors like level resistance against virus strength overall state wellbeingergency room visits needed early diagnosis treatment prevention strategies employed by both patients care providers alike ensuring everyone stays safe while managing this complex condition responsibly over time leading towards eventual eradication efforts targeted toward specific populations segments geographically concentrations where prevalence rates tend higher due various social economic environment

" Nope; no evidence suggests that casually exchanging saliva with an infected person results transmission even without bleeding or cuts present during contact (e g sharing drink utensils). However please remember hygiene practices still apply as usual so keep smiling clean teeth and fresh breath always inside outside workplace home etceteras…) [105398], human message text 'No worries about catchign cold only handshakes allowed indoors' replied back saying they understand how difficult these situations can feel sometimes need reassurance however assure their loved ones will continue supporting each other every step along way till reaching desired outcome becomes reality one day sooner than expected since healthy habbits count too maintain good relations family friendships networks essential component thrivving long term stability welfare happiness achieved successfully passed onto next generation following suit positive cycle sustainability guaranteed endless pursuit excellency fulfille

In [61]:
conversation.run("How does HIV treatment work?")



> Entering new ConversationChain chain...
Prompt after formatting:
As a medical knowledge assistant, my purpose is to deliver precise, succinct information from established medical knowledge. Here is your brief answer:

Current conversation:
[HumanMessage(content='What are the symptoms of HIV soon after infection?'), AIMessage(content=' In some cases when someone has been recently affected especially those who don’t take antire retrovirals promptly once identified might develop acute stage syndrome which includes fever muscle aches joint pains swollen lymph nodes fatigue general malaise headache night sweats chills diarrhea rash last few days followed eventually severe weight loss wasting away until death occurs unless treated appropriately! It serves important reminder why seeking timely professional assistance matters greatly throughout entire process dealing not just life threatenig complications caused directly because you didn want them happen again later down line – getting tes

' Antiviral medications called ART help manage illness by suppressin virus replication prevent damage occurrences within body while also slowiung disease advancement toward full blown condition if left neglected over time leading potentially deadline consequences avoided early intervention ensuring quality longevity for individuals receiving proper care attention paid towards managing conditions effectively minimizing adverse effects associated side treatmenst offering more favorable patient experience overall resulting greater satisfaction wellbeings experienced community wide acknowledging efforts put forth improvisation strengthening bonds between families friends colleagues fostering mutual trust respect beneficial relationships maintained despite challengese encountered on this pathway thus providing strong foundation building harmony peace coexistence amongst all people involved contributing factors facilitatihg societal growth expansion limitlessly expandinf influence beyond bou

In [16]:
conversation.run("Who are considered very close contacts at high risk of meningococcal disease?")



> Entering new ConversationChain chain...
Prompt after formatting:
As a general medical knowledge assistant, I aim to provide accurate and general information based on existing knowledge. However, always consult with a healthcare professional for personalized advice. Based on general medical literature, 

Current conversation:
[HumanMessage(content='Who should people contact if they suspect they is ill?'), AIMessage(content=" Great question! If you or someone you know is experiencing symptoms of schizophrenia, it's essential to seek professional help from a licensed mental health professional. These professionals can provide a proper diagnosis and develop an effective treatment plan tailored to the individual's needs. You can find mental health professionals in your area through your insurance provider, online directories, or by asking for recommendations from your primary care physician. Remember, early intervention and treatment can significantly improve outcomes for individuals wi

" Hi there! As a general medical knowledge assistant, I aim to provide accurate and general information based on existing knowledge. According to general medical literature, individuals who are considered to be very close contacts at high risk of meningococcal disease include:\n* Household contacts of someone with meningococcal disease, especially those who have had direct contact with the affected person's respiratory secretions (e.g., kissing, sharing drinks, or being in close proximity).\n* Healthcare personnel who have provided care to someone with meningococcal disease without using appropriate precautions (e.g., wearing gloves and masks).\n* Anyone who has been exposed to someone with meningococcal disease during the 7-14 days before the onset of symptoms in the affected person.\nIt is important to note that these groups are considered at higher risk because they may be more likely to come into close contact with the bacteria that causes meningococcal disease. However, anyone can